In [1]:
"""dump prophet results to postgres
"""

'dump prophet results to postgres\n'

In [2]:
# import what I need
from sqlalchemy import create_engine
from sqlalchemy_utils import database_exists, create_database
from sqlalchemy import text
import psycopg2
from psycopg2 import sql
import pandas as pd
from psycopg2.extensions import AsIs


In [3]:
# define folder locations, vars
predictions_names = ['ny_prophet_predictions', 'la_prophet_predictions']
folder = './'

dbname = 'produce_predictions'
username = 'donald'

In [4]:
# engine
engine = create_engine('postgres://%s@localhost/%s'%(username,dbname))
print(engine.url)

postgres://donald@localhost/produce_predictions


In [5]:
## create a database (if it doesn't exist)
if not database_exists(engine.url):
    create_database(engine.url)
print(database_exists(engine.url))

True


In [6]:
# now read in each city's predictions - put each in a table in the database
for c in predictions_names:

    predictions = pd.read_csv(folder+c+'.csv', index_col = 0)
    ## insert data into database from Python (proof of concept - this won't be useful for big data, of course)

    predictions.to_sql(c, engine, if_exists='replace')
    
    con = psycopg2.connect(database = dbname, user = username)
# test
sql_query ="""
SELECT "Item", "Current Price", "3 Month Prediction" FROM "ny_prophet_predictions";
"""
query_results=pd.read_sql_query(sql_query,con)
predictions = []
for i in range(0,query_results.shape[0]):
    predictions.append(dict(item=query_results.iloc[i]['Item'], currprice=query_results.iloc[i]['Current Price'], predictprice=query_results.iloc[i]['3 Month Prediction']))
print(predictions)

# test
sql_query ="""
SELECT "Item", "Current Price", "3 Month Prediction" FROM "la_prophet_predictions";
"""
query_results=pd.read_sql_query(sql_query,con)
predictions = []
for i in range(0,query_results.shape[0]):
    predictions.append(dict(item=query_results.iloc[i]['Item'], currprice=query_results.iloc[i]['Current Price'], predictprice=query_results.iloc[i]['3 Month Prediction']))
print(predictions)

con.close()

[{'item': 'APPLES', 'currprice': 28.0, 'predictprice': 35.24}, {'item': 'APRICOTS', 'currprice': 30.21, 'predictprice': 32.21}, {'item': 'ASPARAGUS', 'currprice': 18.16, 'predictprice': 31.17}, {'item': 'AVOCADOS', 'currprice': 29.1, 'predictprice': 55.86}, {'item': 'BANANAS', 'currprice': 16.25, 'predictprice': 15.91}, {'item': 'BEANS', 'currprice': 25.67, 'predictprice': 25.85}, {'item': 'BEETS', 'currprice': 14.07, 'predictprice': 13.47}, {'item': 'BLACKBERRIES', 'currprice': 35.97, 'predictprice': 22.35}, {'item': 'BLUEBERRIES', 'currprice': 25.1, 'predictprice': 21.08}, {'item': 'BROCCOLI', 'currprice': 24.39, 'predictprice': 21.34}, {'item': 'BRUSSELS+SPROUTS', 'currprice': 35.28, 'predictprice': 41.4}, {'item': 'CABBAGE', 'currprice': 14.04, 'predictprice': 12.84}, {'item': 'CANTALOUPS', 'currprice': 14.39, 'predictprice': 13.42}, {'item': 'CAULIFLOWER', 'currprice': 13.26, 'predictprice': 12.63}, {'item': 'CARROTS', 'currprice': 20.79, 'predictprice': 18.14}, {'item': 'CELERY',

In [7]:
# now read in each item's data - put each in a table in the database
citydict = {'NEW+YORK':'newyork', 'LOS+ANGELES':'losangeles'}
producedict = {'APPLES':'apples','APRICOTS':'apricots','ASPARAGUS':'asparagus','AVOCADOS':'avocados',
               'BANANAS':'bananas','BEANS':'beans','BEETS':'beets','BLACKBERRIES':'blackberries',
               'BLUEBERRIES':'blueberries','BROCCOLI':'broccoli','CABBAGE':'cabbage','CANTALOUPS':'cantaloups',
               'CARROTS':'carrots','CAULIFLOWER':'cauliflower','CELERY':'celery','CHERRIES':'cherries',
               'CLEMENTINES':'clementines','CUCUMBERS':'cucumbers','ENDIVE':'endive','GARLIC':'garlic',
               'GINGER+ROOT':'ginger_root','GRAPEFRUIT':'grapefruit','GRAPES':'grapes','HONEYDEWS':'honeydews',
               'KIWIFRUIT':'kiwifruit','LEMONS':'lemons','LETTUCE%2C+ICEBERG':'iceberg_lettuce',
               'LETTUCE%2C+ROMAINE':'romaine_lettuce','LETTUCE%2C+RED+LEAF':'red_lettuce',
               'LETTUCE%2C+GREEN+LEAF':'green_lettuce','LIMES':'limes','NECTARINES':'nectarines',
               'OKRA':'okra','ORANGES':'oranges','PEACHES':'peaches','PEARS':'pears','PEAS+GREEN':'peas',
               'PEPPERS%2C+BELL+TYPE':'bell_peppers','PINEAPPLES':'pineapples','PLUMS':'plums',
               'POTATOES':'potatoes','RADISHES':'radishes','RASPBERRIES':'raspberries','RHUBARB':'rhubarb',
               'SPINACH':'spinach','SQUASH':'squash','STRAWBERRIES':'strawberries',
               'BRUSSELS+SPROUTS':'brussels_sprouts'}

cities = list(citydict.keys())
veggies = list(producedict.keys())

for c in cities:
    for v in veggies:
        data = pd.read_csv(c+'_'+v+'_prophet_output.csv',index_col = 0)
        ## insert data into database from Python (proof of concept - this won't be useful for big data, of course)

        data.to_sql(citydict[c]+'_'+producedict[v], engine, if_exists='replace')
    
# test   
con = psycopg2.connect(database = dbname, user = username)

sql_query ="""
SELECT yhat, yhat_upper FROM "newyork_apples";
"""
query_results=pd.read_sql_query(sql_query,con)
print(query_results)

sql_query2 ="""
SELECT yhat, yhat_upper, ds FROM %(tablename)s;
"""
query_results=pd.read_sql_query(sql_query2,con,params={'tablename':AsIs('newyork_apples')})
print(query_results['ds'])

con.close()

           yhat  yhat_upper
0     24.989577   28.933822
1     25.068215   28.855758
2     25.098412   29.134074
3     25.129793   29.177245
4     25.161824   29.129069
5     25.193953   29.163243
6     25.285358   29.164465
7     25.312380   29.171186
8     25.336862   29.369719
9     25.358381   29.422846
10    25.376576   29.347205
11    25.408699   29.244485
12    25.411498   29.255458
13    25.410361   29.278780
14    25.405443   29.537929
15    25.396997   29.401483
16    25.354475   29.467007
17    25.336357   29.054463
18    25.317365   29.353419
19    25.298268   29.504608
20    25.279889   29.160023
21    25.237942   29.261181
22    25.231434   29.183342
23    25.247223   28.920038
24    25.333685   29.532340
25    25.381583   29.374988
26    25.439953   29.548841
27    25.509209   29.536400
28    25.589659   29.385313
29    25.899509   29.812264
...         ...         ...
2759  37.792176   42.124625
2760  37.944957   41.785868
2761  38.098627   42.017005
2762  38.253205   42